<a href="https://colab.research.google.com/github/danielmachlab/imdb_review_classifier/blob/development/imdb_review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Review Classifier

### Dev Steps
1. Import Dataset from keras/tf
2. View Sentences
3. Translate encoded reviews 
4. Create Model
5. Train Model
6. Test Model
7. Tweak Hyper Parameters


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb

### View Data

In [0]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [0]:
word_index = imdb.get_word_index()
# shift all numbers by three to make room for placeholders
word_index = {k: (v+3) for k, v in word_index.items()} 

# add placeholders to the word_index dictionary
word_index.update({"<PAD>": 0, "<START>": 1, "<UNK>": 2, "<UNUSED>": 3})
list(word_index.items())[-10:]

In [47]:
# view the first review
print('sentence', x_train[0])
print('label', y_train[1])

sentence [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
label 0
